In [1]:
#Installing required Libraries wherever the code is being executed - run only if needed
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-cp37-none-any.whl size=26820 sha256=2f0f9fdac6638a79031ce5c4c2161f3182fba6641881218edf467ce0ffa07a81
  Stored in directory: /root/.cache/pip/wheels/53/6c/18/b87a2e5fa8a82e9c026311de56210b8d1c01846e18a9607fc9
Successfully built pandasql


In [2]:
#Importing required Libraries
import pandas as pd 
import pandasql as psql

In [3]:
# Please remember to navigate to "Shared with me" on your Gdrive home sceen
# Right click on the "HNY_Project" (the primary folder for this project) and select "Add shortcut to Drive"
# This way the below code can find a link to the Project dr folder through your own drive.


# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
# Reading both the Tesseract output files
tess_out_bk_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Tesseract Output/BK_1850_final_output_tesseract.csv')
tess_out_mn_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Tesseract Output/MN_1850_final_output_tesseract.csv')

In [ ]:
# Adding a identifier column based on the dataset they origin from
tess_out_bk_1850["dataset_id"] = 'BK_1850'
tess_out_mn_1850["dataset_id"] = 'MN_1850'

# Appending both dataframes 
frames = [tess_out_bk_1850,tess_out_mn_1850]
tess_out_1850= pd.concat(frames)

In [ ]:
# Creating a column with a 6-digit unique ID per row based on the row index
# Warning: This is being done with no verification of duplication
tess_out_1850['cd_unique_id'] = (tess_out_1850.index+1)+100000

In [ ]:
# Reading both the Adress Standardized files
stand_out_bk_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/BK_1850_with_address2.csv')
stand_out_mn_1850 = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/mn_1850_for_rerun.csv')

In [ ]:
#Exploring the data read above
psql.sqldf("select * from tess_out_1850 LIMIT 10")

,Unnamed: 0,Full Name,First Name,Last Name,Middle Name,Widow-flag,Widow Of,Title,Occupation,Full Address,H Status-flag,H Address,H House Number,H Street Name,H City,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City,Index,Full Entry,dataset_id,cd_unique_id
0,0,Abberly Richard,Richard,Abberly,None,0,None,None,shoemaker,168 Hudson av,Assume,168 Hudson av,168.0,168 Hudson av,None,None,None,None,None,None,None,BK_1850,100001
1,1,Abberly Samuel,Samuel,Abberly,None,0,None,None,None,mason Dean n Vanderbilt av,Assume,mason Dean n Vanderbilt av,NaN,None,"['mason', 'Dean', 'n', 'Vanderbilt', 'av']",None,None,None,None,None,None,BK_1850,100002
2,2,Abbey A C,A,Abbey,C,0,None,None,carpenter,82 Poplar,Assume,82 Poplar,82.0,82 Poplar,None,None,None,None,None,None,None,BK_1850,100003
3,3,"Aessy Horatio G , Columbia Institute for the",Horatio,Aessy,G,0,None,None,None,education of,Assume,education of,NaN,None,"['education', 'of']",None,None,None,None,None,None,BK_1850,100004
4,4,Abbey WS,WS,Abbey,None,0,None,None,None,merchant N Â¥ h Union n Court,Assume,merchant N Â¥ h Union n Court,NaN,None,"['merchant', 'N', 'Â¥', 'h', 'Union', 'n', 'Co...",None,None,None,None,None,None,BK_1850,100005
5,5,Abbott Abraham,Abraham,Abbott,None,0,None,None,marble sawyer,Pacific n Boerum,Assume,Pacific n Boerum,NaN,None,"['Pacific', 'n', 'Boerum']",None,None,None,None,None,None,BK_1850,100006
6,6,Abbott Daniel,Daniel,Abbott,None,0,None,None,None,25 Front . [Court,Assume,25 Front . [Court,25.0,25 Front . [Court,None,None,None,None,None,None,None,BK_1850,100007
7,7,Abbott Francis H,Francis,Abbott,H,0,None,None,merchant,141 Front N Â¥ h Joyalemonn,Assume,141 Front N Â¥ h Joyalemonn,141.0,141 Front N Â¥ h Joyalemonn,None,None,None,None,None,None,None,BK_1850,100008
8,8,Abbott H B,H,Abbott,B,0,None,None,milkman,Kent ay north,Assume,Kent ay north,NaN,None,"['Kent', 'ay', 'north']",None,None,None,None,None,None,BK_1850,100009
9,9,Abbott James,James,Abbott,None,0,None,None,None,laborer Bond c State / [Hoyt,Assume,laborer Bond c State / [Hoyt,NaN,None,"['laborer', 'Bond', 'c', 'State', '/', '[Hoyt']",None,None,None,None,None,None,BK_1850,100010


In [ ]:
#Exploring the data read above
psql.sqldf("select * from stand_out_bk_1850 LIMIT 10")

,Unnamed: 0,Full Name,First Name,Last Name,Middle Name,Widow-flag,Widow Of,Title,Occupation,Full Address,H Status-flag,H Address,H House Number,H Street Name,H City,Address 2,Address 2 House Number,Address 2 Street Name,Address 2 City,Index,Full Entry,hn_1,best_match,street_type,base_function,suffix_true,both_true,address2_hn_1,address2_best_match,address2_street_type,address2_base_function,address2_suffix_true,address2_both_true
0,0,Abberly Richard,Richard,Abberly,None,0,None,None,shoemaker,168 Hudson av,Assume,168 Hudson av,168.0,168 Hudson av,None,None,None,None,None,None,None,168.0,HUDSON AVE,AVE,168 HUDSON AVE,168 HUDSON AVE,168 HUDSON AVE,NaN,None,ST,None,None,None
1,1,Abberly Samuel,Samuel,Abberly,None,0,None,None,None,mason Dean n Vanderbilt av,Assume,mason Dean n Vanderbilt av,NaN,None,"['mason', 'Dean', 'n', 'Vanderbilt', 'av']",None,None,None,None,None,None,NaN,DEAN AND VANDERBILT AVE,AVE,DEAN AND VANDERBILT AVE,DEAN AND VANDERBILT AVE,DEAN AND VANDERBILT AVE,NaN,None,ST,None,None,None
2,2,Abbey A C,A,Abbey,C,0,None,None,carpenter,82 Poplar,Assume,82 Poplar,82.0,82 Poplar,None,None,None,None,None,None,None,82.0,POPLAR,ST,82 POPLAR ST,82 POPLAR ST,82 POPLAR ST,NaN,None,ST,None,None,None
3,3,"Aessy Horatio G , Columbia Institute for the",Horatio,Aessy,G,0,None,None,None,education of,Assume,education of,NaN,None,"['education', 'of']",None,None,None,None,None,None,NaN,EDUCATION OF,ST,EDUCATION OF ST,EDUCATION OF ST,EDUCATION OF ST,NaN,None,ST,None,None,None
4,4,Abbey WS,WS,Abbey,None,0,None,None,None,merchant N Â¥ h Union n Court,Assume,merchant N Â¥ h Union n Court,NaN,None,"['merchant', 'N', 'Â¥', 'h', 'Union', 'n', 'Co...",None,None,None,None,None,None,NaN,UNION AND CT,CT,UNION AND CT,UNION AND CT,UNION AND CT,NaN,MERCHANT,ST,MERCHANT ST,MERCHANT ST,MERCHANT ST
5,5,Abbott Abraham,Abraham,Abbott,None,0,None,None,marble sawyer,Pacific n Boerum,Assume,Pacific n Boerum,NaN,None,"['Pacific', 'n', 'Boerum']",None,None,None,None,None,None,NaN,PACIFIC AND BOERUM,ST,PACIFIC AND BOERUM ST,PACIFIC AND BOERUM ST,PACIFIC AND BOERUM ST,NaN,None,ST,None,None,None
6,6,Abbott Daniel,Daniel,Abbott,None,0,None,None,None,25 Front . [Court,Assume,25 Front . [Court,25.0,25 Front . [Court,None,None,None,None,None,None,None,25.0,FRONT,ST,25 FRONT ST,25 FRONT ST,25 FRONT ST,NaN,CT,ST,CT ST,CT ST,CT ST
7,7,Abbott Francis H,Francis,Abbott,H,0,None,None,merchant,141 Front N Â¥ h Joyalemonn,Assume,141 Front N Â¥ h Joyalemonn,141.0,141 Front N Â¥ h Joyalemonn,None,None,None,None,None,None,None,NaN,JOYALEMONN,ST,JOYALEMONN ST,JOYALEMONN ST,JOYALEMONN ST,141.0,FRONT,ST,141 FRONT ST,141 FRONT ST,141 FRONT ST
8,8,Abbott H B,H,Abbott,B,0,None,None,milkman,Kent ay north,Assume,Kent ay north,NaN,None,"['Kent', 'ay', 'north']",None,None,None,None,None,None,NaN,KENT ALY NORTH,ALY,KENT ALY NORTH ALY,KENT ALY NORTH ALY,KENT ALY NORTH ALY,NaN,None,ST,None,None,None
9,9,Abbott James,James,Abbott,None,0,None,None,None,laborer Bond c State / [Hoyt,Assume,laborer Bond c State / [Hoyt,NaN,None,"['laborer', 'Bond', 'c', 'State', '/', '[Hoyt']",None,None,None,None,None,None,NaN,BOND AND STATE AND HOYT,ST,BOND AND STATE AND HOYT ST,BOND AND STATE AND HOYT ST,BOND AND STATE AND HOYT ST,NaN,None,ST,None,None,None


## Removing White Spaces

In [ ]:
# Using Strip function to remove preceeding and following whitespaces
df1['Full Name'] = df1['Full Name'].str.strip()
df1['First Name'] = df1['First Name'].str.strip()
df1['Last Name'] = df1['Last Name'].str.strip()``
df1['Middle Name'] = df1['Middle Name'].str.strip()

## Handling Titles in First Names

In [ ]:

with open(outputFileStep2, 'rb') as f:
    d = pickle.load(f)

df1_original = pd.read_excel(outputFileStep1) 

print(len(df1)-len(df1_original)) # Should be zero

df1_final_1 = df1_original[['First Name','Last Name ','Middle Name ','Full Name']]

print(len(df1_final_1))


df1_final_1['Full Name'] = df1['Full Name']

if workflow_for == 'Full Name':
  outputFileStep5 = inputFile.replace('.csv','')
  df1_final_1.to_csv(outputFileStep5)

elif workflow_for == 'Full Name':  
  outputFileStep5 = inputFileOriginal2.replace('.csv','') 
  df1_final_1.to_csv(outputFileStep5)




## Removing the part of name after comma


In [ ]:
for i in range(0,len(part_after_comma)):
      if temp[0:2].find(part_after_comma[i])>=0:
          temp = temp[0:2].replace(part_after_comma[i],'') + temp[2:]
  
# Create instersections for near and corner cases
words_after_c = temp.split()[location_of_h+1:]
try:
    location_of_n = words_after_h.index('n')
    intersection_n_street1 = ' '.join(words_after[0:location_of_n])
    intersection_n_street2 = ' '.join(words_after[location_of_n+1:])
